# (Extract part) ETL: Cómo obtener todos los datos de todos los restaurantes en Rappi

En síntesis, este notebook es el primero de 3 que explican cómo obtener en .csv:
<ol>
<li>Los países disponibles en Rappi: paises_rappi.csv</li>
<li>Los cadenas de restaurantes disponibles en cada país: cadenas_restaurantes_df.csv</li>
<li>Las sucursales de cada cadena junto con sus atributos.</li>
<li>Una tabla aparte listando las opiniones ligadas a cada sucursal: opiniones_sucursales.csv</li>
</ol>
------
Nota: Si decides ejecutar este código:
<ol>
<li>Instala las librerías listadas en requirements.txt</li>
<li>Este código utiliza web scraping (Podría relantizar tu conexión a Internet.)</li>
<li>Ten en cuenta que se usa multiprocessing para acelerar ciertas partes del ETL (Podría relantizar tu máquina.)</li>
<li>Ligado al punto anterior, Estas mismas partes me demoraron entre 2 y 4 horas a terminar (Mi CPU tiene 4 cores y tengo una conexión de 230 megas/s).</li>
</ol>
<p>Nota personal: Aún falta enlistar bien las urls con error.</p>
Fecha de edición: 4/12/2022


## Obtenemos paises_rappi.csv

Importamos las librerías a utilizar durante esta parte del proyecto:

In [ ]:
import concurrent.futures
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import re
from IPython.display import clear_output

Debido a que usaremos scrapping con requests, definamos una pequeña función que corrobore si la conexión se logró o surgió un error y, de estar todo okey, nos retorne la sopa.

In [ ]:
def obtener_sopa(url):
    url_request = requests.get(url)
    if url_request.status_code == 200:
        soup = BeautifulSoup(url_request.text, 'lxml')
        return soup
    else:
        status = f'Hubo un problema con la url: {url}'
        print(status)
        return None

A la fecha de esta edición, Rappi tiene todas las url's de cada país listadas en su footer. Con esto en mente obtendremos la etiqueta de los elementos que contienen los href's y los nombres de cada país.

In [ ]:
# Definamos la url base
url_main = "https://www.rappi.com"
# Hacemos el request, verificamos status y obtenemos sopa
sopa = obtener_sopa(url_main) 

Para listar todos los países de Rappi actualmente:
1) Encontraremos el elemento con tag *ul* y id *1* que lista justo lo que necesitamos, a este le llamaremos *elemento_clave*.
2) Luego obtendremos todos los elementos con etiqueta *a* dentro de *elemento_clave*. (En está etiqueta están las url's y los nombres de cada país)
2) Luego listaremos cada url y cada nombre de cada país en *paises_url* y *paises_nombres*
3) Imprimimos resultados

In [ ]:
elemento_clave = sopa.find('ul', id="1") # Primer paso
elementos_a = elemento_clave.find_all('a') # Segundo paso
paises_url = [elementos_a[i].get('href') for i in range(len(elementos_a))] # Tercer paso
paises_nombres = [elementos_a[i].text for i in range(len(elementos_a))] # Sus nombres
paises_url # Ya adivinaste jaajaj

Listamos nuestros datos como df para exportarlo a .csv después

In [ ]:
# Creamos el DF
paises_df = pd.DataFrame({"url_paises": paises_url, "nombre_pais": paises_nombres})
# Revisemos su estado
paises_df.head(3)

Ahora, a la fecha de esta edición, listaremos el tipo de cambio en doláres para cada moneda latinoamericana. (Esto nos servirá para convertir los precios de cada catálogo en una sola currency).

In [ ]:
# Creamos la columna de intercambio de equivalencia a 1000 dólares.
equivalente_10000_dolares = [
    1676508,# Argentina
    52195,# Brasil
    8835000,# Chile
    47680700,# Colombia
    5981305, # Costa rica
    10000, # Ecuador
    193914,# Mexico
    38223.49,# Perú
    394049.80 # Uruguay
]
paises_df["1000_dollars_exchange"] = equivalente_10000_dolares
# Revisemos su estado
paises_df.head(3)

Listo. Ahora solo exportamos y guardamos el .csv

In [ ]:
paises_df.to_csv("paises_rappi.csv", index_label="id_pais")

## Obteniendo cadenas_restaurantes_df.csv

<p>Para nuestra suerte, Rappi ya ha indexado cada cadena de restaurantes de cada país en un solo catálogo. El mismo dividido en distintas páginas (de la A hasta la Z junto con las cadenas que empiezan con números).
<p>Para acceder a este, solo se debe agregar "/catalogo/restaurants/a-1" a la url de cada país.
<p>Ejemplo, para Ecuador sería: "https://www.rappi.com.ec/catalogo/restaurants/a-a1".

Para lograr nuestro objetivo realizaremos 2 cosas:
1) Listar todas las subpáginas de cada catálogo de cada país.
2) Obtener los nombres y url's de cada cadena de cada subpágina.

Para esto necesitaremos nuestro df de países.

In [ ]:
paises_df = pd.read_csv("paises_rappi.csv")

### Listar todas las subpáginas de cada catálogo de cada país.

In [ ]:
# Nuestra url base
url_inicial_catalogo = "/catalogo/restaurants/a-1"
id_pais_subcatalogos_all = []
urls_catalogos_all = []
for i in paises_df.index:
    url_pais_elegido = paises_df.url_paises[i][:-1] #Obtenemos la url de cada país quitando 
                                                    # -el último '/'
    url_catalogo_pais = url_pais_elegido + url_inicial_catalogo
    try:
        catalogo_soup = obtener_sopa(url_catalogo_pais) # Obtenemos la sopa
        if sopa != None: # Si no hubo ningun problema al obtener la sopa
            # Obtenemos todos los subcatálogos
            paginas_catalogo = catalogo_soup.find_all(class_="sc-39328323-1 jiuMaW")
            # Obtenemos cada url de cada subcatálogo y lo concatenamos todo
            urls_subcatalogos = [url_pais_elegido + paginas_catalogo[j].get("href")
                                  for j
                                  in range(len(paginas_catalogo))]
            urls_catalogos_all = urls_catalogos_all + urls_subcatalogos
            # Igualmente con el id del país correspondiente
            id_pais_subcatalogos_all = id_pais_subcatalogos_all + [i
                                                               for j
                                                               in range(len(urls_subcatalogos))]
            clear_output(wait=True)
            print(f"Porcentaje completado: {round((i+1)*100 / len(paises_df.index), 2)}")
    except:
        print(f"No se pudo obtener el catalogo de: {url_catalogo_pais}")
        continue
print("Terminado")

In [ ]:
# Creamos el DF
subcatalogos_df = pd.DataFrame({"url_subcatalogos": urls_catalogos_all, "id_pais": id_pais_subcatalogos_all})
# Revisemos su estado
subcatalogos_df.head(3)

In [ ]:
subcatalogos_df.to_csv("subcatalogos.csv",index_label="id_subcatalogo")

### Obtener los nombres y url's de cada cadena de cada subpágina.
Para esto usaremos las url's obtenidas anteriormente.

In [ ]:
subcatalogos_df = pd.read_csv("subcatalogos.csv")
# Revisamos su estado
subcatalogos_df.head(3)

<p>En la interfaz encontramos que las *urls* y los *nombres* de los restaurantes podemos obtenerlos mediante la clase "sc-bdfBQB eXopiF sc-iqHYmW gcZftM secondary". Con esto en mente corremos el siguiente código:</p>
---------
<p>Nota: A partir de aquí utilizaremos multiprocessing para acelerar las cosas, de modo que dejaremos de agrupar nuestros datos en columnas sino en filas</p>

In [ ]:
problema_url = []
def proccess_subcatalogos(index_to_process):
    i = index_to_process
    url_cadena_resturantes = []
    nombre_cadena_resturantes = []
    id_pais = []
    
    url_subcatalogo = subcatalogos_df.url_subcatalogos[i] # Obtenemos la url
    sub_id_pais = subcatalogos_df.id_pais[i] # Además del id del país
    sopa = obtener_sopa(url_subcatalogo) # Obtenemos la sopa
    # Trabajamos la sopa
    try:
        if sopa != None: # Si no hubo ningun problema al obtener la sopa
            # Obtenemos todos los nombres y urls y las concatenamos
            elementos_cadena_resturantes = sopa.find_all(class_="sc-bdfBQB eXopiF sc-iqHYmW gcZftM secondary")
            url_cadena_resturantes = url_cadena_resturantes + [elementos_cadena_resturantes[j].get("href")
                                                              for j
                                                              in range(len(elementos_cadena_resturantes))]
            nombre_cadena_resturantes = nombre_cadena_resturantes + [elementos_cadena_resturantes[j].find_all("span")[0].text
                                                                     for j 
                                                                    in range(len(elementos_cadena_resturantes))]
            # Luego guardamos el id del país correspondiente
            id_pais = id_pais + [sub_id_pais
                               for j
                               in range(len(nombre_cadena_resturantes))]
            clear_output(wait=True)
            print(f"Porcentaje completado: {round((i+1)*100 / len(subcatalogos_df.index), 2)}")
    except:
        print(f"No se pudo las sucursales en: {url_subcatalogo}")
        problema_url.append(url_subcatalogo)
    return (url_cadena_resturantes, nombre_cadena_resturantes, id_pais)

In [ ]:
# Aquí usamos concurrent y multiproccesing para acelerar nuestro tiempo
all_index_subcatalogos = subcatalogos_df.index
# Obtenemos los datos
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(proccess_subcatalogos, all_index_subcatalogos)
# Guardamos nuestra data
data = list(results)

Revisamos si obtuvimos algún error en alguna url

In [ ]:
problema_url

Cool, ninguno. Ahora nuestra series están guardadas en subtuplas de sublistas de la lista data. Para poder utilizarla tenemos que descomprimir estas tuplas y luego concatenarlas

In [ ]:
# Creamos pd.Series vacías para usarlas en la concatenación
url_cadena_resturantes = pd.Series([], dtype=str)
nombre_cadena_resturantes = pd.Series([], dtype=str)
id_pais = pd.Series([], dtype="int64")

# Concatenamos cada lista con su respectiva serie
for tuplita in data:
    url_cadena_resturantes_next = pd.Series(tuplita[0])
    url_cadena_resturantes = pd.concat([url_cadena_resturantes_next,url_cadena_resturantes])
    
    nombre_cadena_resturantes_next = pd.Series(tuplita[1])
    nombre_cadena_resturantes = pd.concat([nombre_cadena_resturantes_next,nombre_cadena_resturantes])
    
    id_pais_next = pd.Series(tuplita[2])
    id_pais = pd.concat([id_pais_next,id_pais])

Cool, todo okey. Ahora veamos como quedo:

In [ ]:
cadenas_restaurantes_df = pd.DataFrame({"nombre_cadena": nombre_cadena_resturantes,
                                       "url_cadena": url_cadena_resturantes,
                                       "id_pais": id_pais})
# Lo revisamos
cadenas_restaurantes_df.head(3)

In [ ]:
# Reseteamos el índice
cadenas_restaurantes_df.reset_index(drop=True, inplace=True)
# Veamos como queda finalmente
cadenas_restaurantes_df.head(3)

In [ ]:
# Guardamos nuestro trabajo
cadenas_restaurantes_df.to_csv("cadenas_restaurantes.csv", index_label="id_cadena")

## Sucursales: Obteniendo todas las sucursales de cada cadena

Cool, a 1 paso de completar nuestra extracción. Ahora, en cada cadena de restaurantes existen diferentes sucursales y cada una de ellas con un nombre, una url y una dirección únicas).
Ahora tocaría:
1) Aplicar la misma idea del paso anterior pero para cada sucursal.

In [ ]:
# Importamos nuestro trabajo anterior
cadenas_restaurantes_df = pd.read_csv("cadenas_restaurantes.csv") 
# Lo revisamos
cadenas_restaurantes_df.head(3) 

In [ ]:
problema_url = []
no_hay_sucursales_url = []
def proccess_sucursales(index_to_process):
    i = index_to_process
    url_sucursales = []
    nombre_sucursales = []
    direccion_sucursales = []
    id_cadena = []
    
    url_cadena_elegida = cadenas_restaurantes_df.url_cadena[i] # Obtenemos la url
    sub_id_cadena = cadenas_restaurantes_df.id_cadena[i] # Además del id del país
    sopa = obtener_sopa(url_cadena_elegida) # Obtenemos la sopa
    # Trabajamos la sopa
    try:
        if sopa != None: # Si no hubo ningun problema al obtener la sopa
            div_sucursales = sopa.find("div",
                              {'data-testid': 'topRestCard'},
                              class_="sc-9fb51c13-6 fIgfiC"
                              )
            try:
                sucursales = div_sucursales.find_all("a")
                # Obtenemos todos los nombres y urls y las concatenamos
                url_pais_base = re.split("/restaurantes" , url_cadena_elegida)[0]
                # Obtenemos los atributos de cada sucursal
                url_sucursales = url_sucursales + [url_pais_base + sucursales[i].get("href")
                                                   for i
                                                   in range(len(sucursales))]
                nombre_sucursales = nombre_sucursales + [sucursales[i].find_all("h3")[0].text
                                                        for i
                                                         in range(len(sucursales))]
                direccion_sucursales = direccion_sucursales + [sucursales[i].find_all("div", class_="sc-bxivhb fFeDyp sc-d9669f19-7 iOdleX")[0].text
                                                               for i
                                                               in range(len(sucursales))]
                # Luego guardamos el id del país correspondiente
                id_cadena = id_cadena + [sub_id_cadena
                                   for j
                                   in range(len(nombre_sucursales))]
                clear_output(wait=True)
                print(f"Porcentaje completado: {round((i+1)*100 / len(cadenas_restaurantes_df.index), 2)}")
            except:
                print(f"No hay sucursales en la url {url_cadena_elegida}.")
                no_hay_sucursales_url.append(url_cadena_elegida)
    except:
        print(f"No se pudo ingresar a: {url_cadena_elegida}")
        problema_url.append(url_cadena_elegida)
    return (url_sucursales, nombre_sucursales, direccion_sucursales, id_cadena)

In [ ]:
# Aplicamos multiproccesing
all_index_subcatalogos = cadenas_restaurantes_df.index
# Obtenemos los datos
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(proccess_sucursales, all_index_subcatalogos)
# Guardamos nuestra data
data = list(results)

Revisamos si obtuvimos algún error en alguna url

In [ ]:
print(f"Problemas encontrados urls: {len(problema_url)}")
print(f"Cadenas sin sucursal: {len(no_hay_sucursales_url)}")

En este caso nos lista que no hay errores, esto sucede porque no he puesto las variables en global de modo que sucede esto. Pero como pudimos ver en el Output, sí hay cadenas que o no tienen sucursales disponibles o no se pudo acceder a estas mismas. Por el momento lo dejaremos pasar.

In [ ]:
# Creamos pd.Series vacías para usarlas en la concatenación
url_sucursal = pd.Series([], dtype=str)
nombre_sucursal = pd.Series([], dtype=str)
direccion_sucursal = pd.Series([], dtype=str)
id_cadena = pd.Series([], dtype="int64")

# Concatenamos cada lista con su respectiva serie
for i, tuplita in enumerate(data):
    url_sucursal_next = pd.Series(tuplita[0], dtype=str)
    url_sucursal = pd.concat([url_sucursal_next,url_sucursal])
    
    nombre_sucursal_next = pd.Series(tuplita[1], dtype=str)
    nombre_sucursal = pd.concat([nombre_sucursal_next,nombre_sucursal])
    
    direccion_sucursal_next = pd.Series(tuplita[2], dtype=str)
    direccion_sucursal = pd.concat([direccion_sucursal_next, direccion_sucursal])
    
    id_cadena_next = pd.Series(tuplita[3], dtype="int64")
    id_cadena = pd.concat([id_cadena_next,id_cadena])
    clear_output(wait=True)
    print(f"Porcentaje completado: {round((i+1)*100 / len(data), 2)}")

Bueno, como podrás imaginarte, para este proposito también hay una clase y una etiqueta específica que revisar jaja

In [ ]:
sucursales_df = pd.DataFrame({"id_cadena": id_cadena,
                              "url_sucursal": url_sucursal,
                              "nombre_sucursal": nombre_sucursal,
                             "direccion_sucursal": direccion_sucursal})
# Lo revisamos
sucursales_df.head(3)

In [ ]:
# Ordenamos en base a la id_cadena
sucursales_df = sucursales_df.sort_values("id_cadena", axis=0)
# Reseteamos el índice
sucursales_df.reset_index(drop=True, inplace=True)
# Veamos como queda finalmente
sucursales_df.head(3)

In [ ]:
# Guardamos nuestro trabajo
sucursales_df.to_csv("sucursales.csv", index_label="id_sucursal")

Cool, todo bien. La única nota ha dejar es que existen algunas cadenas de comida a las que no se pudieron sacar sus sucursales. Esto se debe a que SÍ están indexadas en el directorio de Rappi pero NO tienen sucursales disponibles, se han dado de baja o por x razón no aparecen sus sucursales.

## Obteniendo las opiniones, precios y otros atributos de cada sucursal

Con esto ingresaremos a cada sucursal y obtendremos la siguiente información:
- Opiniones: Las opiniones y sus porcentajes de cada sucursal.
- Precios: Todos los precios de cada producto listado en el catálogo.
- Otros atributos: Número de calificaciones, estrellas promedio, tiempo de envío, etc.
----
En algunas no se tienen estos datos, así que tendremos que poner "No abierto al público / No calificado".

In [ ]:
sucursales_df = pd.read_csv("sucursales.csv")
sucursales_df.head(3)

Ahora obtenemos los datos que nos interesan. Probaremos con una sucursal

In [ ]:
# Obtenemos la url de la sucursal de ejemplo
sucursal_url = sucursales_df.url_sucursal[3]
print(f"Obteniendo sopa de: {sucursal_url}")
# Obtenemos la sopa
sopa = obtener_sopa(sucursal_url)

In [ ]:
# Usaremos una lambda para retornar "None" si no hay opiniones o las opiniones si sí las hay.
revisador_de_nonetypes = lambda element: np.nan if element == None else element.text
# Creamos nuestra función para obtener los atributos
def get_all_attr(soup_sucursal):
    try:
        # Obtenemos las estrellas y el número de opiniones
        overral_stars = revisador_de_nonetypes(soup_sucursal.find("span", class_="sc-bxivhb gJCKbU"))
        number_opinions = revisador_de_nonetypes(soup_sucursal.find("span", class_="sc-bxivhb dVvqfA")) # Le quitamos los parentesis
        tiempo_delivery = revisador_de_nonetypes(soup_sucursal.find("span", class_="sc-bxivhb ecrUmJ"))
        tipo_envio = revisador_de_nonetypes(soup_sucursal.find("div", class_="chakra-skeleton css-1vjr0v9"))
        if not pd.isna(number_opinions):
            number_opinions = int(number_opinions[1:-1]) # Para quitar los parentesis y obtener el número
        return [overral_stars, number_opinions, tiempo_delivery, tipo_envio]
    except:
        print("03 Problema al obtener atributos globales. Retornando error")
        error = "Problema atributos"
        return error
get_all_attr(sopa)

In [ ]:
# Nuestra función para obtener todos los precios
def get_all_prices(soup_sucursal):
    try:
        # Encontrar lista de precios de productos
        div_prices_unparsed = soup_sucursal.find_all("span", class_="chakra-text css-kowr8")
        prices = [div_prices_unparsed[i].text
                        for i in range(len(div_prices_unparsed))]
        return prices
    except:
        print("02 Problema al obtener precios. Retornando error")
        prices = "Problema precios"
        return prices
get_all_prices(sopa)

In [ ]:
# Nuestra sopa para obtener las diferentes opiniones con su porcentaje
def get_all_opinions(soup_sucursal):
    try:
        div_opinions_unparsed = soup_sucursal.find_all("div", class_="css-z7mtfw")
        # Nos quedamos con los spans de cada opinion para tener su texto y porcentajes
        opinions_unparsed = [div_opinions_unparsed[i].find_all("span")
                        for i in range(len(div_opinions_unparsed))]
        # Agarramos el texto Y porcentajes
        opinions = [(opinions_unparsed[i][0].text, opinions_unparsed[i][1].text)
                    for i in range(len(opinions_unparsed))]
        return opinions
    except:
        print("01 Problema al obtener opiniones. Retornando error")
        opinions = "Error opiniones"
        return opinions
get_all_opinions(sopa) # Si retorna una lista vacía es que no se encontraron opiniones

Con estos datos podemos empezar a obtener todos los metadatos de cada sucursal

In [ ]:
problema_obtener_url = []
# Definición de nuestra función base
def process_sucursal(index_sucursal):
    i = index_sucursal
    sucursal_url = sucursales_df.url_sucursal[i]
    # Obtenemos el ID de la sucursal
    id_sucursal = sucursales_df.id_sucursal[i] 
    try:
        # Obtenemos la sopa
        sopa = obtener_sopa(sucursal_url)
        attributes = get_all_attr(sopa)
        prices = get_all_prices(sopa)
        opinions = get_all_opinions(sopa)
        print(f"Porcentaje completado: {round((i+1)*100 / len(sucursales_df.index), 2)}")
        return id_sucursal, attributes, prices, opinions
    except:
        try:
            # Obtenemos la sopa
            print("Intentando obtener sopa denuevo")
            sopa = obtener_sopa(sucursal_url)
            # Obtenemos todos los datos
            attributes = get_all_attr(sopa)
            prices = get_all_prices(sopa)
            opinions = get_all_opinions(sopa)
            clear_output(wait=True)
            print(f"Porcentaje completado: {round((i+1)*100 / len(sucursales_df.index), 2)}")
            return id_sucursal, attributes, prices, opinions
        except:
            print(f"Hubo un problema con la sopa de la url {url_sucursal_elegida}")
            problema_obtener_url.append(url_sucursal_elegida)

In [ ]:
# Aquí usamos concurrent y multiproccesing para acelerar nuestro tiempo
all_index_sucursales = sucursales_df.index

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(process_sucursal, all_index_sucursales)
    
# Guardamos la data
data = list(results)

In [ ]:
# Lo ponemos en un df
atributos_sucursales_bruto_df = pd.DataFrame.from_records(data,
                                                    columns = ["id_sucursal",
                                                               "attributes",
                                                               "prices",
                                                               "opinions"])
# Revisemos que tal quedó
atributos_sucursales_bruto_df.head(3)

## Revisión de últimos errores

Busquemos si queda algún error por allí. Por si acaso

In [ ]:
atributos_sucursales_bruto_df[atributos_sucursales_bruto_df["opinions"] == "Error opiniones"]

In [ ]:
# Retorna la lista de índices en donde se detectaron errores
revisar = atributos_sucursales_bruto_df[atributos_sucursales_bruto_df["opinions"] == "Error opiniones"].index
revisar

Veamos si podemos solucionarlo aplicando la función denuevo

In [ ]:
# Reapliquemos 
all_index_sucursales = revisar

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(process_sucursal, all_index_sucursales)
    
# Guardamos la data
data = list(results)

In [ ]:
# Revisemos
atributos_sucursales_bruto_df_errores = pd.DataFrame.from_records(data,
                                                    columns = ["id_sucursal",
                                                               "attributes",
                                                               "prices",
                                                               "opinions"])
# Revisemos que tal quedó
atributos_sucursales_bruto_df_errores

In [ ]:
# Retorna la lista de índices en donde se detectaron errores (denuevo)
revisar_errores = atributos_sucursales_bruto_df_errores[atributos_sucursales_bruto_df_errores["opinions"] == "Error opiniones"].index
len(revisar_errores)

Bien, parece que nos decisimos de los deshicimos de los errores. Reemplazemos los nuevos valores nuevos por los anteriores malos

In [ ]:
# Recambiemos con los datos buenos por los malos en el df original
for i, j in enumerate(atributos_sucursales_bruto_df_errores.id_sucursal):
    new_data = atributos_sucursales_bruto_df_errores[atributos_sucursales_bruto_df_errores["id_sucursal"] == j].loc[i,:]
    atributos_sucursales_bruto_df.iloc[j,:] = new_data

In [ ]:
# Chequeamos que todo haya sido cambiado
atributos_sucursales_bruto_df.loc[revisar,:]

In [ ]:
# Chequeamos por errores una última vez
revisar = atributos_sucursales_bruto_df[atributos_sucursales_bruto_df["opinions"] == "Error opiniones"].index
len(revisar)

Cool, con esto hecho, ordenamos nuestro df y lo envíamos

In [ ]:
# Ordenamos el df
atributos_sucursales_bruto_df = atributos_sucursales_bruto_df.sort_values("id_sucursal")
# Lo guardamos
atributos_sucursales_bruto_df.to_csv("atributos_sucursales_bruto.csv", index=False)

Listo. Hasta aquí acabaría la en E en nuestro ETL, continua a la siguiente parte 'Transform' para conocer como seguimos desde aquí. Gracias por tu tiempo!